###### Importing dependencies

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
from sqlalchemy.dialects.postgresql import insert
import sqlalchemy
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

###### Importing dataset

In [2]:
df = pd.read_csv('superstore.csv')

In [4]:
df.shape

(51290, 27)

In [5]:
#dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51290 non-null  object 
 1   City            51290 non-null  object 
 2   Country         51290 non-null  object 
 3   Customer.ID     51290 non-null  object 
 4   Customer.Name   51290 non-null  object 
 5   Discount        51290 non-null  float64
 6   Market          51290 non-null  object 
 7   记录数             51290 non-null  int64  
 8   Order.Date      51290 non-null  object 
 9   Order.ID        51290 non-null  object 
 10  Order.Priority  51290 non-null  object 
 11  Product.ID      51290 non-null  object 
 12  Product.Name    51290 non-null  object 
 13  Profit          51290 non-null  float64
 14  Quantity        51290 non-null  int64  
 15  Region          51290 non-null  object 
 16  Row.ID          51290 non-null  int64  
 17  Sales           51290 non-null 

### Performing ETL

###### Data Cleaning

In [6]:
#lowering casing
df.columns = df.columns.str.lower().str.replace(r'[\s\.]', '_', regex=True)


In [7]:
df.columns.values

array(['category', 'city', 'country', 'customer_id', 'customer_name',
       'discount', 'market', '记录数', 'order_date', 'order_id',
       'order_priority', 'product_id', 'product_name', 'profit',
       'quantity', 'region', 'row_id', 'sales', 'segment', 'ship_date',
       'ship_mode', 'shipping_cost', 'state', 'sub_category', 'year',
       'market2', 'weeknum'], dtype=object)

In [9]:
#Checking unknown column
df['记录数'].unique()

array([1], dtype=int64)

In [10]:
#The column above contains irrelevant data and so should be dropped
df = df.drop(columns = ['记录数'], axis = 1)

In [11]:
#Checking for missing values
missing_df = df.isnull().sum()
print(missing_df)

category          0
city              0
country           0
customer_id       0
customer_name     0
discount          0
market            0
order_date        0
order_id          0
order_priority    0
product_id        0
product_name      0
profit            0
quantity          0
region            0
row_id            0
sales             0
segment           0
ship_date         0
ship_mode         0
shipping_cost     0
state             0
sub_category      0
year              0
market2           0
weeknum           0
dtype: int64


There are no missing values in the dataset

In [14]:
#Checking for duploicated values
duplicated_df = df.duplicated().sum()
print(duplicated_df)

0


There are no duplicated values in the dataset

In [16]:
#checking data types
type_df = df.dtypes
print(type_df)

category           object
city               object
country            object
customer_id        object
customer_name      object
discount          float64
market             object
order_date         object
order_id           object
order_priority     object
product_id         object
product_name       object
profit            float64
quantity            int64
region             object
row_id              int64
sales               int64
segment            object
ship_date          object
ship_mode          object
shipping_cost     float64
state              object
sub_category       object
year                int64
market2            object
weeknum             int64
dtype: object


In [17]:
#Converting dates
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])

# Dropping duplicate orders
df = df.drop_duplicates(subset=['order_id', 'product_id', 'customer_id'])


In [18]:
#checking market
df['market'] is df['market2']

False

In [20]:
df['market'].unique()

array(['US', 'EU', 'LATAM', 'Africa', 'APAC', 'EMEA', 'Canada'],
      dtype=object)

In [21]:
df['market2'].unique()

array(['North America', 'EU', 'LATAM', 'Africa', 'APAC', 'EMEA'],
      dtype=object)

In [22]:
#dropping market2 since it's contains market 1
df = df.drop(columns = ['market2'], axis = 1)

In [23]:
df.columns.values

array(['category', 'city', 'country', 'customer_id', 'customer_name',
       'discount', 'market', 'order_date', 'order_id', 'order_priority',
       'product_id', 'product_name', 'profit', 'quantity', 'region',
       'row_id', 'sales', 'segment', 'ship_date', 'ship_mode',
       'shipping_cost', 'state', 'sub_category', 'year', 'weeknum'],
      dtype=object)

### Extracting And Transforming Data

In [25]:
#dim_date
dates = pd.concat([df['order_date'], df['ship_date']]).drop_duplicates().reset_index(drop=True)
dim_date = pd.DataFrame({
    'date_id': range(1, len(dates)+1),
    'date': dates,
})
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['day'] = dim_date['date'].dt.day
dim_date['quarter'] = dim_date['date'].dt.quarter
dim_date['week_num'] = dim_date['date'].dt.isocalendar().week


In [26]:
#dim_customer
dim_customer = df[['customer_id','customer_name','segment',
                   'city','state','country','region']].drop_duplicates()


In [27]:
#dim_products
dim_product = df[['product_id','product_name','category',
                  'sub_category']].drop_duplicates()

In [28]:
#dim_ship
dim_ship = df[['ship_mode','shipping_cost',
               'order_priority']].drop_duplicates()

###### Fact Table

In [29]:
# Mapping order_date and ship_date to date_id
date_mapping = dim_date.set_index('date')['date_id'].to_dict()
df['order_date_id'] = df['order_date'].map(date_mapping)
df['ship_date_id'] = df['ship_date'].map(date_mapping)

#Creating fact_sales
fact_sales = df[['order_id','order_date_id','ship_date_id','customer_id',
                 'product_id','ship_mode','sales','profit','quantity',
                 'discount','shipping_cost']].copy()


# Loading

### Database Connection

In [30]:
#Connection parameters
username = 'postgres'          
password = 'Best2410#'      
host = 'localhost'         
port = '5432'              
database = 'store_warehouse' 

#Creating engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


In [51]:
from sqlalchemy.dialects.postgresql import insert
from sqlalchemy import MetaData, Table

def upsert_table(df, table_name, key_columns, engine):
    """
    Efficiently inserts DataFrame rows into PostgreSQL using ON CONFLICT DO NOTHING.
    key_columns should be a list of columns forming the unique constraint.
    """
    # Drop duplicates based on the key columns
    df = df.drop_duplicates(subset=key_columns)

    metadata = MetaData()
    table = Table(table_name, metadata, autoload_with=engine)

    with engine.begin() as conn:
        stmt = insert(table).on_conflict_do_nothing(index_elements=key_columns)
        result = conn.execute(stmt, df.to_dict(orient='records'))

        inserted = result.rowcount or 0
        skipped = len(df) - inserted

    print(f"{table_name}: {inserted} rows inserted, {skipped} skipped.")


In [55]:
# Dimensions first
#upsert_table(dim_date, 'dim_date', 'date_id', engine)
upsert_table(dim_customer, 'dim_customer', ['customer_id'], engine)
upsert_table(dim_product, 'dim_product', ['product_id'], engine)
upsert_table(dim_ship, 'dim_ship', ['ship_mode'], engine)
upsert_table(dim_date, 'dim_date', ['date_id'], engine)

# Composite key (if needed)
upsert_table(fact_sales, 'fact_sales', ['order_id','product_id'], engine)


dim_customer: 0 rows inserted, 4873 skipped.
dim_product: 0 rows inserted, 10292 skipped.
dim_ship: 0 rows inserted, 4 skipped.
dim_date: 0 rows inserted, 1468 skipped.
fact_sales: 0 rows inserted, 51252 skipped.


In [56]:
#checking fact_table
query = "SELECT * FROM fact_sales;"
df_fact = pd.read_sql(query, engine)
df_fact.head()

,fact_id,order_id,order_date_id,ship_date_id,customer_id,product_id,ship_mode,sales,profit,quantity,discount,shipping_cost
0,1,CA-2011-130813,1,1414,LS-172304,OFF-PA-10002005,Second Class,19.0,9.3312,3,0.0,4.37
1,2,CA-2011-148614,2,979,MV-174854,OFF-PA-10002893,Standard Class,19.0,9.2928,2,0.0,0.94
2,3,CA-2011-118962,3,352,CS-121304,OFF-PA-10000659,Standard Class,21.0,9.8418,3,0.0,1.81
3,4,CA-2011-118962,3,352,CS-121304,OFF-PA-10001144,Standard Class,111.0,53.2608,2,0.0,4.59
4,5,CA-2011-146969,4,1177,AP-109154,OFF-PA-10002105,Standard Class,6.0,3.1104,1,0.0,1.32


In [57]:
df_fact.shape

(51252, 12)

In [58]:
#checking customer_table
query = "SELECT * FROM dim_customer;"
df_cus = pd.read_sql(query, engine)
df_cus.head()

,customer_id,customer_name,segment,city,state,country,region
0,LS-172304,Lycoris Saunders,Consumer,Los Angeles,California,United States,West
1,MV-174854,Mark Van Huff,Consumer,Los Angeles,California,United States,West
2,CS-121304,Chad Sievert,Consumer,Los Angeles,California,United States,West
3,AP-109154,Arthur Prichep,Consumer,Los Angeles,California,United States,West
4,JF-154904,Jeremy Farry,Consumer,Los Angeles,California,United States,West


In [59]:
df_cus.shape

(4873, 7)

In [61]:
#checking product
query = "SELECT * FROM dim_product"
df_pro = pd.read_sql(query, engine)
df_pro.head()

,product_id,product_name,category,sub_category
0,OFF-PA-10002005,Xerox 225,Office Supplies,Paper
1,OFF-PA-10002893,"Wirebound Service Call Books, 5 1/2"" x 4""",Office Supplies,Paper
2,OFF-PA-10000659,"Adams Phone Message Book, Professional, 400 Me...",Office Supplies,Paper
3,OFF-PA-10001144,Xerox 1913,Office Supplies,Paper
4,OFF-PA-10002105,Xerox 223,Office Supplies,Paper


In [62]:
df_pro.shape

(10292, 4)

In [63]:
#checking ship
query = "SELECT * FROM dim_ship"
df_ship = pd.read_sql(query, engine)
df_ship.head()

,ship_mode,shipping_cost,order_priority
0,Second Class,4.37,High
1,Standard Class,0.94,Medium
2,Same Day,2.39,Critical
3,First Class,1.77,Medium


In [67]:
df_ship.shape

(4, 3)

In [65]:
#checking dim_date
query = "SELECT * FROM dim_date"
df_date = pd.read_sql(query, engine)
df_date.head()

,date_id,date,year,month,day,quarter,week_num
0,1,2011-01-07,2011,1,7,1,1
1,2,2011-01-21,2011,1,21,1,3
2,3,2011-08-05,2011,8,5,3,31
3,4,2011-09-29,2011,9,29,3,39
4,5,2011-10-19,2011,10,19,4,42


In [66]:
df_date.shape

(1468, 7)

###### Validation

In [69]:
#Check a few rows in fact_sales
query = "SELECT * FROM fact_sales LIMIT 10;"
df_val = pd.read_sql(query, engine)
df_val

,fact_id,order_id,order_date_id,ship_date_id,customer_id,product_id,ship_mode,sales,profit,quantity,discount,shipping_cost
0,1,CA-2011-130813,1,1414,LS-172304,OFF-PA-10002005,Second Class,19.0,9.3312,3,0.0,4.37
1,2,CA-2011-148614,2,979,MV-174854,OFF-PA-10002893,Standard Class,19.0,9.2928,2,0.0,0.94
2,3,CA-2011-118962,3,352,CS-121304,OFF-PA-10000659,Standard Class,21.0,9.8418,3,0.0,1.81
3,4,CA-2011-118962,3,352,CS-121304,OFF-PA-10001144,Standard Class,111.0,53.2608,2,0.0,4.59
4,5,CA-2011-146969,4,1177,AP-109154,OFF-PA-10002105,Standard Class,6.0,3.1104,1,0.0,1.32
5,6,CA-2011-117317,5,5,JF-154904,OFF-PA-10004519,Same Day,13.0,6.5856,3,0.0,2.39
6,7,CA-2011-125829,6,68,WB-218504,OFF-PA-10000223,Standard Class,19.0,9.3312,3,0.0,1.15
7,8,CA-2011-151295,7,154,JA-159704,OFF-PA-10001947,Standard Class,12.0,5.8604,2,0.0,0.92
8,9,CA-2011-135090,8,328,SP-209204,OFF-PA-10002245,Standard Class,54.0,24.2190,9,0.0,6.98
9,10,CA-2011-133830,9,609,RL-196154,OFF-PA-10001363,Standard Class,49.0,23.0864,4,0.0,1.99


In [73]:
#Check foreign key relationships
query = """
SELECT COUNT(*) 
FROM fact_sales f
LEFT JOIN dim_customer c 
    ON f.customer_id = c.customer_id
WHERE c.customer_id IS NOT NULL;
"""

df_joins = pd.read_sql(query, engine)
df_joins


,count
0,51252


In [75]:
# Check product join
query = """
SELECT COUNT(*) 
FROM fact_sales f
LEFT JOIN dim_product p 
    ON f.product_id = p.product_id
WHERE p.product_id IS NOT NULL;
"""
df_p = pd.read_sql(query, engine)
df_p

,count
0,51252


In [78]:
# Check date join
query = """
SELECT COUNT(*) AS missing_order_dates
FROM fact_sales f
LEFT JOIN dim_date d 
    ON f.order_date_id = d.date_id
WHERE d.date_id IS NOT NULL;
"""
df_d =pd.read_sql(query, engine)
df_d

,missing_order_dates
0,51252


In [80]:
query = """
SELECT COUNT(*) AS missing_ship_dates
FROM fact_sales f
LEFT JOIN dim_date d 
    ON f.ship_date_id = d.date_id
WHERE d.date_id IS NULL;
"""
df = pd.read_sql(query, engine)
df

,missing_ship_dates
0,0


In [81]:
#all relationships
query = """
SELECT 
    (SELECT COUNT(*) FROM fact_sales) AS fact_rows,
    (SELECT COUNT(*) FROM fact_sales f 
     JOIN dim_customer c ON f.customer_id = c.customer_id) AS valid_customers,
    (SELECT COUNT(*) FROM fact_sales f 
     JOIN dim_product p ON f.product_id = p.product_id) AS valid_products,
    (SELECT COUNT(*) FROM fact_sales f 
     JOIN dim_ship s ON f.ship_mode = s.ship_mode) AS valid_ships,
    (SELECT COUNT(*) FROM fact_sales f 
     JOIN dim_date d ON f.order_date_id = d.date_id) AS valid_order_dates,
    (SELECT COUNT(*) FROM fact_sales f 
     JOIN dim_date d ON f.ship_date_id = d.date_id) AS valid_ship_dates;
"""

df = pd.read_sql(query, engine)
df

,fact_rows,valid_customers,valid_products,valid_ships,valid_order_dates,valid_ship_dates
0,51252,51252,51252,51252,51252,51252
